# Lab 02. 로지스틱 희귀 앙상블 실습
---

- 앙상블 기법은 실무에서 거의 않쓴다.
- 인공지능 대회에서 마지막에 정확도를 끌어올리기 위해 주로 사용한다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 데이터셋, 데이터 로더 생성

In [2]:
dataset = FashionMNIST(root="./data", train=True, transform=ToTensor(), download=False)
train_set, val_set = train_test_split(dataset, test_size=0.1, random_state=777)

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(val_set, batch_size=100, shuffle=False)

## 모델 선언

In [3]:
class LogisticRegression(nn.Module) : 
    def __init__(self, input_size, num_classes) : 
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x) : 
        out = self.linear(x)
        
        return out

## 하이퍼 파라미터 설정 

In [4]:
input_size = 28 * 28 
num_classes = 10
num_epoch = 100
lr =  0.001
num_models = 5 # 앙상블에 사용할 모델 개수

## 앙상블 

In [5]:
models = [LogisticRegression(input_size, num_classes) for _ in range(num_models)]

print(models)

[LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)]


## 모델, 손실 함수, 옵티마이저 정의

In [6]:
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=lr) for model in models]

print(optimizers)

[SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)]


## Train Loop

In [7]:
for epoch in range(num_epoch) : 
    for i, (images, labels) in enumerate(train_loader) :
        # 데이터 로드 
        images = images.reshape(-1, input_size)
        labels = labels
        
        # 순전파 및 손실 계산 
        for j in range(num_models) : 
            outputs = models[j](images)
            loss = criterion(outputs, labels)
            
            # 역전파 및 가중치 업데이트
            optimizers[j].zero_grad()
            loss.backward()
            optimizers[j].step()
            
    # 검증 코드 추가 #
    with torch.no_grad() : 
        total, correct = 0,0 
        for images, labels in test_loader : 
            images = images.reshape(-1, input_size)
            """
            outputs = torch.zeros(images.size()[0], num_classes)
            이미지 배치에 대한 출력 텐서 초기화 
            >> 후속 단계에서 이미지에 대한 예측값 업데이트 가능
            """
            outputs = torch.zeros(images.size()[0], num_classes)
            # 앙상블 모델의 예측값 더하기 
            for j in range(num_models) : 
                outputs += models[j](images)
                
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        val_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epoch}], Val ACC >> {val_acc:.2f}%")


Epoch [1/100], Val ACC >> 66.33%
Epoch [2/100], Val ACC >> 67.27%
Epoch [3/100], Val ACC >> 68.28%
Epoch [4/100], Val ACC >> 69.27%
Epoch [5/100], Val ACC >> 70.37%
Epoch [6/100], Val ACC >> 71.97%
Epoch [7/100], Val ACC >> 73.10%
Epoch [8/100], Val ACC >> 74.15%
Epoch [9/100], Val ACC >> 74.73%
Epoch [10/100], Val ACC >> 75.23%
Epoch [11/100], Val ACC >> 75.85%
Epoch [12/100], Val ACC >> 76.65%
Epoch [13/100], Val ACC >> 77.13%
Epoch [14/100], Val ACC >> 77.33%
Epoch [15/100], Val ACC >> 77.60%
Epoch [16/100], Val ACC >> 77.83%
Epoch [17/100], Val ACC >> 78.10%
Epoch [18/100], Val ACC >> 78.22%
Epoch [19/100], Val ACC >> 78.65%
Epoch [20/100], Val ACC >> 78.73%
Epoch [21/100], Val ACC >> 78.88%
Epoch [22/100], Val ACC >> 79.08%
Epoch [23/100], Val ACC >> 79.27%
Epoch [24/100], Val ACC >> 79.33%
Epoch [25/100], Val ACC >> 79.33%
Epoch [26/100], Val ACC >> 79.55%
Epoch [27/100], Val ACC >> 79.90%
Epoch [28/100], Val ACC >> 79.98%
Epoch [29/100], Val ACC >> 80.07%
Epoch [30/100], Val ACC